In [6]:
import os

from pathlib import Path

from sqlmodel import Session, select

In [2]:
os.chdir(Path.cwd().parent)

In [3]:
Path.cwd()

PosixPath('/Users/jochen/projects/fastdeploy')

In [4]:
from app import database

from app.models import Step

# Start Deployment

What should start deployment do?

* Create a deployment model with created = now() and finished = None timestamps
* Look up the steps from last deployment and recreate them as "pending" with new deployment id
* If there are no last steps, create one new dummy step
* Start deployment task
* Mark the first step as "running"

# Finish Step

What should finish step do?

* If the currently running step != the finished step, mark the currently running step as pending again and create a new finished step
* If the currently running step == the fninshed step, mark the currently running step as finished and set the next pending step to "running"

# Finish Deployment

What should happen on finish deployment?

* If the deployment has failed, do nothing
* If the deployment was successful, remove all pending steps

# Find Last Successful Deployment

In [38]:
with Session(database.engine) as session:
    stmt = """
        select distinct(step.deployment_id)
          from step, deployment
            on step.deployment_id=deployment.id
         where
           deployment.service_id = 4
           and step.state != 'success'
         group by step.deployment_id
        having count(step.id) > 1
    """
    result = session.exec(stmt).all()

In [62]:
with Session(database.engine) as session:
    stmt = """
        select step.deployment_id
          from deployment, step
         where step.deployment_id=deployment.id
           and step.state == "success"
           and deployment.service_id = 3
           and step.deployment_id not in (
            select distinct(step.deployment_id)
              from step, deployment
                on step.deployment_id=deployment.id
             where deployment.service_id = 3
               and step.state != 'success'
             group by step.deployment_id
            having count(step.id) > 1
           )
         group by step.deployment_id
        having count(step.id) > 0
         order by deployment.created desc
    """
    deployment_id = session.exec(stmt).all()[0][0]
    result = session.exec(stmt).all()

In [56]:
with Session(database.engine) as session:
    stmt = """
        select step.deployment_id
          from deployment, step
         where step.deployment_id=deployment.id
           and step.state == "success"
           and deployment.service_id = 3
         group by step.deployment_id
        having count(step.id) > 0
    """
    deployment_id = session.exec(stmt).all()[0][0]

In [54]:
len(result)

2

In [63]:
deployment_id

2

In [79]:
with Session(database.engine) as session:
    stmt = """
        select step.deployment_id
          from deployment, step
         where step.deployment_id=deployment.id
           and step.state == "success"
           and deployment.service_id = :service_id
           and step.deployment_id not in (
            select distinct(step.deployment_id)
              from step, deployment
                on step.deployment_id=deployment.id
             where deployment.service_id = :service_id
               and step.state != 'success'
             group by step.deployment_id
            having count(step.id) > 1
           )
         group by step.deployment_id
        having count(step.id) > 0
         order by deployment.created desc
    """
    deployment_id = session.exec(stmt, params={"service_id": 3}).all()[0][0]

print(deployment_id)

2


In [65]:
with Session(database.engine) as session:
    help(session.exec)


Help on method exec in module sqlmodel.orm.session:

exec(statement: Union[sqlmodel.sql.expression.Select, sqlmodel.sql.expression.SelectOfScalar, sqlmodel.sql.base.Executable[~_TSelectParam]], *, params: Union[Mapping[str, Any], Sequence[Mapping[str, Any]], NoneType] = None, execution_options: Mapping[str, Any] = immutabledict({}), bind_arguments: Optional[Mapping[str, Any]] = None, _parent_execute_state: Optional[Any] = None, _add_event: Optional[Any] = None, **kw: Any) -> Union[sqlmodel.engine.result.Result[~_TSelectParam], sqlmodel.engine.result.ScalarResult[~_TSelectParam]] method of sqlmodel.orm.session.Session instance

